In [1]:
import boto3
from pii_masker import PIIMasker
from main import combined_names, organizations, locations
s3_client = boto3.client('s3')
objects = s3_client.list_objects_v2(Bucket='mai-support-test', Prefix='processed_data_4kTextfiles/')

In [5]:
objects["Contents"]

[{'Key': 'processed_data_4kTextfiles/Processed01ForewordEspanol.mp3.txt',
  'LastModified': datetime.datetime(2024, 1, 25, 7, 8, 40, tzinfo=tzutc()),
  'ETag': '"7e419e2527e6c730ca3cbadcc4bc7a97"',
  'Size': 1557,
  'StorageClass': 'STANDARD'},
 {'Key': 'processed_data_4kTextfiles/Processed01GregWTheDoctorsOpinion.mp3.txt',
  'LastModified': datetime.datetime(2024, 1, 25, 7, 8, 41, tzinfo=tzutc()),
  'ETag': '"faaa55050cf2c3662f6917a9a7dec648"',
  'Size': 26005,
  'StorageClass': 'STANDARD'},
 {'Key': 'processed_data_4kTextfiles/Processed01JimPBigBookWorkshopEufaulaAL9820131.mp3.txt',
  'LastModified': datetime.datetime(2024, 1, 25, 7, 8, 42, tzinfo=tzutc()),
  'ETag': '"0ff194980e23acefe584e5fc290fc3a9"',
  'Size': 46444,
  'StorageClass': 'STANDARD'},
 {'Key': 'processed_data_4kTextfiles/Processed01TheBookStudyIntroductionAAHistory1BillmeetsEbbyT.mp3.txt',
  'LastModified': datetime.datetime(2024, 1, 25, 7, 8, 43, tzinfo=tzutc()),
  'ETag': '"bf5b8706d63e994328f7bdbfbbd312a3"',
  'Si

In [4]:
len(objects["Contents"])

1000

In [7]:
import boto3

# Replace 'your_access_key' and 'your_secret_key' with your actual AWS credentials
bucket_name = 'mai-support-test'
folder_name = 'processed_data_4kTextfiles/'

# Create an S3 client
s3 = boto3.client('s3')

# List all objects in the specified folder of the bucket
response = s3.list_objects(Bucket=bucket_name, Prefix=folder_name)

# Extract filenames (keys) from the response
filenames = [obj['Key'] for obj in response.get('Contents', [])]
len(filenames)
# Print the filenames
# for filename in filenames:
#     print(filename)


1000

In [9]:
import boto3

def list_all_objects(bucket_name, folder_name):
    s3 = boto3.client('s3')

    all_filenames = []

    # Initial list_objects request
    response = s3.list_objects(Bucket=bucket_name, Prefix=folder_name)

    # Extract filenames (keys) from the response
    filenames = [obj['Key'] for obj in response.get('Contents', [])]
    all_filenames.extend(filenames)

    # Continue paginating until there are no more results
    while response.get('IsTruncated', False):
        # Use the marker from the previous response to get the next set of results
        marker = response['Contents'][-1]['Key']
        response = s3.list_objects(Bucket=bucket_name, Prefix=folder_name, Marker=marker)
        
        # Extract filenames (keys) from the response
        filenames = [obj['Key'] for obj in response.get('Contents', [])]
        all_filenames.extend(filenames)

    return all_filenames

# Replace 'your_access_key' and 'your_secret_key' with your actual AWS credentials
bucket_name = 'mai-support-test'
folder_name = 'processed_data_4kTextfiles/'

filenames = list_all_objects(bucket_name, folder_name)
# Print the filenames
for filename in filenames:
    print(filename)


processed_data_4kTextfiles/Processed01ForewordEspanol.mp3.txt
processed_data_4kTextfiles/Processed01GregWTheDoctorsOpinion.mp3.txt
processed_data_4kTextfiles/Processed01JimPBigBookWorkshopEufaulaAL9820131.mp3.txt
processed_data_4kTextfiles/Processed01TheBookStudyIntroductionAAHistory1BillmeetsEbbyT.mp3.txt
processed_data_4kTextfiles/Processed01sandyboaf20081.mp3.txt
processed_data_4kTextfiles/Processed02JoeCharlieAAHistory2OxfordGroup.mp3.txt
processed_data_4kTextfiles/Processed02KerryCFridayNightShare.mp3.txt
processed_data_4kTextfiles/Processed02TheDoctorsOpinionEspanol.mp3.txt
processed_data_4kTextfiles/Processed03AAHistory3andTheOrganizationoftheBigBook.mp3.txt
processed_data_4kTextfiles/Processed03BillsStoryEspanol.mp3.txt
processed_data_4kTextfiles/Processed04AAHistory4.mp3.txt
processed_data_4kTextfiles/Processed04JimP.BigBookWorkshopEufaulaAL.91520134.mp3.txt
processed_data_4kTextfiles/Processed04ThereisasolutionEspanol.mp3.txt
processed_data_4kTextfiles/Processed05JimPBigBookW

In [ ]:
import boto3
from pii_masker import PIIMasker
from main import combined_names, organizations, locations

def process_and_upload_files(s3_client, source_bucket, source_folder, target_folder):
    # List all files in the source folder
    objects = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    if 'Contents' in objects:
        for obj in objects['Contents']:
            file_key = obj['Key']
            # Check if the object is a file and not just a folder name
            if file_key.endswith('/'):
                continue
            
            # Get the file content
            file_obj = s3_client.get_object(Bucket=source_bucket, Key=file_key)
            file_content = file_obj['Body'].read().decode('utf-8')

            # Define new file name and path
            new_file_name = file_key.split('/')[-1].replace('.txt', '_pii.txt')
            new_file_key = f"{target_folder}/{new_file_name}"

            masker = PIIMasker()
            masker.set_fake_names(combined_names)
            masker.set_fake_orgs(organizations)
            masker.set_fake_locations(locations)
            masked_text = masker.mask_all_pii(file_content)
            # print(masked_text)
            # print("**************")
            # print("Mapping:", masker.get_mapping())

            # Upload the file to the new location with the new name
            s3_client.put_object(Bucket=source_bucket, Key=new_file_key, Body=masked_text)

            print(f"File {file_key} processed and uploaded as {new_file_key}")

def main():
    # Define your bucket and folders
    source_bucket = 'mai-support-test'
    source_folder = 'processed_data_4kTextfiles/'
    target_folder = 'pii4k'

    # Initialize S3 client
    s3_client = boto3.client('s3')

    # Process and upload files
    process_and_upload_files(s3_client, source_bucket, source_folder, target_folder)

if __name__ == "__main__":
    main()

    #277,278
